# Analyzing failed SQL Server agent jobs

Run this query to retrieve information on failed jobs.

In [1]:
-- List info on all failed SQL Server agent jobs

select j.name                                                                                  as JobName,
       h.step_name                                                                             as StepName,
       CONCAT_WS(' ', CONVERT(char(10), CAST(STR(h.run_date, 8, 0) AS datetime), 111),
                 STUFF(STUFF(RIGHT('000000' +
                                   CAST(h.run_time AS varchar(6)), 6), 5, 0, ':'), 3, 0, ':')) as RunDateTime,
       h.run_duration                                                                          as StepDuration,
       case h.run_status
           when 0 then 'Failed'
           when 1 then 'Succeeded'
           when 2 then 'Retry'
           when 3 then 'Cancelled'
           when 4 then 'In Progress'
       end                                                                                     as ExecutionStatus,
       h.message                                                                               as MessageGenerated
from msdb.dbo.sysjobhistory h
    inner join msdb.dbo.sysjobs j
    on j.job_id = h.job_id
where h.run_status = 0
order by h.run_date desc, h.run_time desc;

(0 rows affected)

Total execution time: 00:00:00.080

JobName,StepName,RunDateTime,StepDuration,ExecutionStatus,MessageGenerated


This query lists all jobs and their schedules.

In [2]:
-- List SQL Server jobs and schedules

select j.name          as [Job name],
       t.step_id       as [Step ID],
       j.description   as [Description],
       t.command       as [Command],
       t.database_name as [Database name],
       case
           when h.freq_type = 1 then 'One time only'
           when h.freq_type = 4 then 'Daily'
           when h.freq_type = 8 then 'Weekly'
           when h.freq_type = 16 then 'Monthly'
           else 'Other'
       end             as [Frequency type],
       h.freq_interval as [Interval],
       dateadd(second,
               (v.last_run_time / 10000 * 3600) +
               (((v.last_run_time % 10000 - v.last_run_time % 100) / 100) * 60) +
               (v.last_run_time % 100),
               convert(datetime, cast(nullif(v.last_run_date, 0) as nvarchar(10))))
                       as [Last run datetime],
       dateadd(second,
               (s.next_run_time / 10000 * 3600) +
               (((s.next_run_time % 10000 - s.next_run_time % 100) / 100) * 60) +
               (s.next_run_time % 100),
               convert(datetime, cast(nullif(s.next_run_date, 0) as nvarchar(10))))
                       as [Next run datetime]
from msdb.dbo.sysjobs j
    inner join msdb.dbo.sysjobschedules s
    on j.job_id = s.job_id
    left join msdb.dbo.sysjobsteps t
    on j.job_id = t.job_id
    left join msdb.dbo.sysschedules h
    on s.schedule_id = h.schedule_id
    left join msdb.dbo.sysjobservers v
    on j.job_id = v.job_id
where j.enabled = 1
order by 1, 2;

(34 rows affected)

Total execution time: 00:00:00.107

Job name,Step ID,Description,Command,Database name,Frequency type,Interval,Last run datetime,Next run datetime
Adopt_A_Stream_Draft_Event_Deletions,1,on 3/7/2022 this job is delpoyed from Dev .83 Server by Azmath Fatima after the request from Sampath Karre.,"/********* SCRIPT FOR Deleting the EVENT IDS IN THE ADOPT-A-STREAM *****************************/ USE [AdoptAStream] GO /***************************************************************************** Author: Vidyanand Dhande Overview: Cursor to delete certain events from the tables Modification History: When Who What ---------- ------------------- ---------------------------------------- 7-14-2021 Vidhu Script for Adopt-A-Stream Event ID deletions *******************************************************************************/ BEGIN DECLARE @Event_rid int; DECLARE EventID_cursor CURSOR FOR SELECT event_rid FROM [AdoptAStream].dbo.app_event WHERE Draft_Ind = 1 AND Cast(createddate AS DATE) <= CAST(GETDATE() - 7 AS DATE) OPEN EventID_cursor; BEGIN TRANSACTION BEGIN TRY FETCH NEXT FROM EventID_cursor into @Event_rid; WHILE @@FETCH_STATUS = 0 BEGIN DELETE FROM [dbo].[evt_bacterial_detail] WHERE event_rid = @Event_rid DELETE FROM [dbo].[evt_stream_habitat_detail] WHERE event_rid = @Event_rid DELETE FROM [dbo].[evt_miv] WHERE [miv_detail_rid] IN ( SELECT [miv_detail_rid] FROM [dbo].[evt_miv_detail] WHERE event_rid = @Event_rid ) DELETE FROM [dbo].[evt_miv_detail] WHERE event_rid = @Event_rid DELETE FROM [dbo].[evt_chemical] WHERE [chem_detail_rid] IN ( SELECT [chem_detail_rid] FROM [dbo].[evt_chemical_detail] WHERE event_rid = @Event_rid ) DELETE FROM [dbo].[evt_chemical_detail] WHERE event_rid = @Event_rid DELETE FROM aas_app_file WHERE file_rid IN ( SELECT file_rid FROM aas_evt_file WHERE aas_event_rid = @Event_rid ) DELETE FROM aas_evt_file WHERE aas_event_rid = @Event_rid DELETE FROM [dbo].[evt_audit] WHERE event_rid = @Event_rid DELETE FROM [dbo].[con_event] WHERE event_rid = @Event_rid DELETE FROM [dbo].[app_event] WHERE event_rid = @Event_rid FETCH next from EventID_cursor into @Event_rid; END; COMMIT TRANSACTION; END TRY BEGIN CATCH IF @@trancount>0 Rollback transaction; DECLARE @ErrorNumber INT = ERROR_NUMBER(); DECLARE @ErrorLine INT = ERROR_LINE(); DECLARE @ErrorMessage NVARCHAR(4000) = ERROR_MESSAGE(); DECLARE @ErrorSeverity INT = ERROR_SEVERITY(); DECLARE @ErrorState INT = ERROR_STATE(); PRINT 'Actual error number: ' + CAST(@ErrorNumber AS VARCHAR(10)); PRINT 'Actual line number: ' + CAST(@ErrorLine AS VARCHAR(10)); RAISERROR(@ErrorMessage, @ErrorSeverity, @ErrorState); END CATCH; CLOSE EventID_cursor; DEALLOCATE EventID_cursor; END",master,Daily,1,2024-03-15 04:00:00.000,2024-03-16 04:00:00.000
AIRBRANCH_DeactivateInactiveIaipUsers,1,No description available.,USE [AIRBRANCH] GO DECLARE @RC int -- TODO: Set parameter values here. EXECUTE @RC = [dbo].[DeactivateInactiveIaipUsers] GO,master,Monthly,1,2024-03-01 01:00:00.000,2024-04-01 01:00:00.000
AIRBRANCH_ICIS_STAGE_ALL,1,No description available.,USE [AIRBRANCH] GO DECLARE @RC int -- TODO: Set parameter values here. EXECUTE @RC = [etl].[ICIS_Stage_All] GO,master,Daily,1,2024-03-14 21:00:00.000,2024-03-15 21:00:00.000
AIRBRANCH_PD_EIS_DATA_ANNUAL,1,No description available.,USE [AIRBRANCH] GO DECLARE @RC int -- TODO: Set parameter values here. EXECUTE @RC = [dbo].[PD_EIS_DATA_ANNUAL] GO,master,Monthly,1,2024-01-01 04:00:00.000,2025-01-01 04:00:00.000
AIRBRANCH_PD_FEE_ADMIN,1,This job schedule to run PD_FEE_ADMIN from AIRBRANCH,USE [AIRBRANCH] GO DECLARE @RC int -- TODO: Set parameter values here. EXECUTE @RC = [dbo].[PD_FEE_ADMIN] GO,master,Monthly,1,2024-01-01 04:00:00.000,2025-01-01 04:00:00.000
AIRBRANCH_USP_IAIP_TO_STG,1,No description available.,USE [AIRBRANCH] GO DECLARE @RC int -- TODO: Set parameter values here. EXECUTE @RC = [etl].[USP_IAIP_TO_STG] GO,master,Daily,1,2024-03-15 00:00:00.000,2024-03-16 00:00:00.000
Clean up txt files.Subplan_1,1,No description available.,"/Server ""$(ESCAPE_NONE(SRVR))"" /SQL 